#### Realizado por: 
#### Desiderio Nevado López
#### Laura Martín Olinero 
#### Daniel Quesada Pérez de San Roman

### Antes se realizó, en un código aparte la partición del CSV. Después se transpasó tanto los datos originales, como la carpeta con los 102 CSV a Google Drive.


In [ ]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null

In [ ]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practica_Spark').getOrCreate()

In [ ]:
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = spark.read.csv('/content/drive/MyDrive/iot_telemetry_data.csv', header=True, inferSchema=True)

In [ ]:
!head /content/drive/MyDrive/iot_telemetry_data.csv

"ts","device","co","humidity","light","lpg","motion","smoke","temp"
"1.5945120943859746E9","b8:27:eb:bf:9d:51","0.004955938648391245","51.0","false","0.00765082227055719","false","0.02041127012241292","22.7"
"1.5945120947355676E9","00:0f:00:70:91:0a","0.0028400886071015706","76.0","false","0.005114383400977071","false","0.013274836704851536","19.700000762939453"
"1.5945120980735729E9","b8:27:eb:bf:9d:51","0.004976012340421658","50.9","false","0.007673227406398091","false","0.02047512557617824","22.6"
"1.594512099589146E9","1c:bf:ce:15:ec:4d","0.004403026829699689","76.80000305175781","true","0.007023337145877314","false","0.018628225377018803","27.0"
"1.594512101761235E9","b8:27:eb:bf:9d:51","0.004967363641908952","50.9","false","0.007663577282372411","false","0.020447620810233658","22.6"
"1.5945121044684107E9","1c:bf:ce:15:ec:4d","0.004391003954583357","77.9000015258789","true","0.007009458543138704","false","0.01858890754005078","27.0"
"1.5945121054488637E9","b8:27:eb:bf:9d:51","0.00

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
creditsSchema = StructType([
    StructField('ts', TimestampType()),
    StructField('device', StringType()),
    StructField('co', DoubleType()),
    StructField('humidity', DoubleType()),
    StructField('light', BooleanType()),
    StructField('lpg', DoubleType()),
    StructField('motion', BooleanType()),
    StructField('smoke', DoubleType()),
    StructField('temp', DoubleType()),
])

In [ ]:
datos = (spark.read.option("header", "true")
           .option("quote", "\"")  # Tuning escape double quotes
           .option("escape", "\"") # Tuning escape double quotes
           .csv('/content/drive/MyDrive/iot_telemetry_data.csv', schema=creditsSchema)
          )

In [ ]:
import pyspark
pyspark.__version__

'3.4.1'

In [ ]:
datos.printSchema()

root
 |-- ts: timestamp (nullable = true)
 |-- device: string (nullable = true)
 |-- co: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- light: boolean (nullable = true)
 |-- lpg: double (nullable = true)
 |-- motion: boolean (nullable = true)
 |-- smoke: double (nullable = true)
 |-- temp: double (nullable = true)



In [ ]:
datos.show(10)

+----+-----------------+--------------------+-----------------+-----+--------------------+------+--------------------+------------------+
|  ts|           device|                  co|         humidity|light|                 lpg|motion|               smoke|              temp|
+----+-----------------+--------------------+-----------------+-----+--------------------+------+--------------------+------------------+
|null|b8:27:eb:bf:9d:51|0.004955938648391245|             51.0|false| 0.00765082227055719| false| 0.02041127012241292|              22.7|
|null|00:0f:00:70:91:0a|0.002840088607101...|             76.0|false|0.005114383400977071| false|0.013274836704851536|19.700000762939453|
|null|b8:27:eb:bf:9d:51|0.004976012340421658|             50.9|false|0.007673227406398091| false| 0.02047512557617824|              22.6|
|null|1c:bf:ce:15:ec:4d|0.004403026829699689|76.80000305175781| true|0.007023337145877314| false|0.018628225377018803|              27.0|
|null|b8:27:eb:bf:9d:51|0.00496736

In [ ]:
# Número total de filas
print("Número total de filas:", datos.count())

# Número total de columnas
print("Número total de columnas:", len(datos.columns))

Número total de filas: 405184
Número total de columnas: 9


Los datos están formados por 405.184 filas con 9 columnas. En la carpeta "Salidas" tenemos 102 CSV con 4.000 filas por archivo y 1 encabezado, excepto el último archivo que tiene 1.184 filas

# APARTADO 1
## Consulta 1
### Para cada batch, calcula el valor máximo y mínimo registrado de concentración de monóxido de carbono y concentración de gas petróleo licuado.

Este código utiliza PySpark, una biblioteca para el procesamiento distribuido de datos en clústeres de computadoras, para realizar algunas operaciones en archivos CSV. El objetivo es obtener los valores máximos y mínimos de concentración de monóxido de carbono (CO) y gas petróleo licuado (GPL) de los datos presentes en los archivos CSV

In [ ]:
import glob
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType

# Creamos la sesión de spark
spark = SparkSession.builder.appName("MaxMinValues").getOrCreate()

# Ruta de la carpeta donde se encuentran los archivos CSV generados
csv_folder_path = '/content/drive/MyDrive/Salida'

# Obtenemos la lista de los resultados generados
file_list = glob.glob(csv_folder_path + "/*.csv")

# Iterar sobre cada archivo CSV
for file_path in file_list:
    datos = spark.read.csv(file_path, header=True, schema=creditsSchema)

    # Valor máximo y mínimo de concentración de monóxido de carbono
    max_co = datos.agg({"co": "max"}).collect()[0][0]
    min_co = datos.agg({"co": "min"}).collect()[0][0]

    # Valor máximo y mínimo de concentración de gas petróleo licuado
    max_gpl = datos.agg({"lpg": "max"}).collect()[0][0]
    min_gpl = datos.agg({"lpg": "min"}).collect()[0][0]

    print(f"Archivo: {file_path}")
    print(f"Concentración de monóxido de carbono - Máximo: {max_co}, Mínimo: {min_co}")
    print(f"Concentración de gas petróleo licuado - Máximo: {max_gpl}, Mínimo: {min_gpl}")
    print("-------------------------------------------")

Archivo: /content/drive/MyDrive/Salida/Archivo-0000.csv
Concentración de monóxido de carbono - Máximo: 0.005119798892464754, Mínimo: 0.0022640150403052165
Concentración de gas petróleo licuado - Máximo: 0.007832991067433015, Mínimo: 0.004340824874553169
-------------------------------------------
Archivo: /content/drive/MyDrive/Salida/Archivo-0001.csv
Concentración de monóxido de carbono - Máximo: 0.0049516728589025436, Mínimo: 0.002300243760707829
Concentración de gas petróleo licuado - Máximo: 0.007646057802883879, Mínimo: 0.004390963374389081
-------------------------------------------
Archivo: /content/drive/MyDrive/Salida/Archivo-0002.csv
Concentración de monóxido de carbono - Máximo: 0.004905171957408317, Mínimo: 0.0024534975370069504
Concentración de gas petróleo licuado - Máximo: 0.007594046947029975, Mínimo: 0.004600697135977176
-------------------------------------------
Archivo: /content/drive/MyDrive/Salida/Archivo-0003.csv
Concentración de monóxido de carbono - Máximo: 0.0

Con el código anterior, buscamos los valores máximos y mínimos de concentración de monóxido de carbono y gas petróleo licuado en cada archivo, y muestra los resultados en la consola, indicando qué archivo se procesó y cuáles fueron los valores máximos y mínimos encontrados para cada sustancia.